# Advanced Examples
(not yet final)

Some extra files not provided in this repo are necessary!
See ``example_data/__init__.py`` for details on how to provide them

In [ ]:
import libfmp.b as lfb
import numpy as np
import matplotlib.pyplot as plt
import cv2
from makeplotplayable import Session

from example_data import *

sr = 48000
audio_a, _ = lfb.read_audio(simple_audio_file, mono=True, Fs=sr)

def load_av_example_data():
    WIDTH = 48 * 10
    HEIGHT = 36 * 10

    def frame2array(f):
        img = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (WIDTH, HEIGHT))
        return np.array(img, dtype=np.uint8)

    video_capture = cv2.VideoCapture(video_file)
    arrays = list()
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        array = frame2array(frame)
        arrays.append(array)

    array = np.stack(arrays, axis=0)

    audio, _ = lfb.read_audio(audio_to_video_file, Fs=sr, mono=True)

    return audio * 0.8, array


audio_b, video = load_av_example_data()

# Example 1:
This example shows navigation between multiple audio files in one Session.
It will loop and alternate between files.
The time encodes the specific version,
all versions are mapped linearly (in a real application these mappings would be more complicated)

In [ ]:
# combine audio clips into one to allow Session to play it
combined_audio = np.concatenate((audio_a, audio_b))

session01 = Session(combined_audio, sr, looping=True)
session01.start()


# noinspection PyShadowingNames
@session01
def plot(x, sr, durations, index, name):
    start_times = np.zeros_like(durations)
    for i in range(durations.shape[0] - 1):
        start_times[i + 1] = start_times[i] + durations[i]

    fig, ax, line = lfb.plot_signal(x, sr)

    # map the time of all clips to a position
    def time_to_pos(time_):
        j = np.searchsorted(start_times, time_) - 1
        time_ -= start_times[j]
        progress = time_ / durations[j]
        pos = progress * durations[index]
        return pos

    return fig, ax, {"title": name,
                     "custom_time_to_pos_function": time_to_pos,
                     "custom_pos_to_time_function": lambda pos: pos + start_times[
                         index]}  # map to time depending on start time


durations = np.array([audio_a.shape[0] / sr, audio_b.shape[0] / sr])

# it is possible to use different plotting functions
plot(audio_a, sr, durations, 0, 'C01 A')
plot(audio_b, sr, durations, 1, 'C01 B')

# Example 2:
This example shows that animating a plot even allows for video playback

In [ ]:
session02 = Session(audio_b, sr, looping=True)
session02.start()

# noinspection PyShadowingNames
@session02
def plot_audio(x, sr):
    fig, ax, line = lfb.plot_signal(x, sr)

    #return the figure and the axis for the cursor as a tuple
    return fig, ax, {"title": "C02 Audio for Video"}


plot_audio(audio_b, sr)


# noinspection PyShadowingNames
@session02
def plot_video(video):
    fig = plt.figure()
    ax = plt.subplot(1, 1, 1)

    mat_plot = ax.imshow(np.zeros_like(video[0]), cmap='gray', vmin=0, vmax=255, interpolation='antialiased')

    last_index = 0

    # draw the correct frame (pos is frame index)
    def draw_function(time_, pos, paused):
        nonlocal last_index
        # don't redraw if the frame has not changed
        if last_index == pos:
            return False

        mat_plot.set(data=video[pos])
        last_index = pos
        return True

    # implement play/pause functionality
    pp_pressed = False

    def on_key(event):
        nonlocal pp_pressed
        if event.key in [" ", "enter"]:
            pp_pressed = True

    fig.canvas.mpl_connect('key_press_event', on_key)

    def update_func(time_, pos, paused):
        nonlocal pp_pressed
        new_paused = paused ^ pp_pressed
        pp_pressed = False
        return time_, new_paused

    return fig, ax, {"title": "C02 Video",
                     "draw_function": draw_function,
                     "artists": [mat_plot],
                     "override_update_function": update_func,  # implement play/pause functionality + disable cursor
                     "custom_time_to_pos_function": lambda time: int(
                         min(video.shape[0] - 1, time * 30))}  # calculate the frame index


plot_video(video)

# Example 3:
Test streaming of a long audio file

In [ ]:
session03 = Session.from_file(long_audio_file)
session03.start()


# noinspection PyShadowingNames
@session03
def plot(duration):
    fig, ax = plt.subplots()

    ax.plot(np.array([0, duration/3600]))
    ax.set_xlabel("number of files played")
    ax.set_ylabel("time in hours")

    return fig, ax, {"title": "C03 Streaming Example"}


plot(session03.duration)

# Example 4:
Test stereo playback, each ear will receive a different song

In [ ]:
combined_audio = np.stack((audio_a[:sr * 10], audio_b[:sr * 10]), axis=0)

session04 = Session(combined_audio, sr)
session04.start()


# noinspection PyShadowingNames
@session04
def plot(audio, sr, title):
    fig, ax, line = lfb.plot_signal(audio, sr)

    return fig, ax, {"title": title}


plot(combined_audio[0], sr, "C04 Left")
plot(combined_audio[1], sr, "C04 Right")